## WARNING: run this script will overwrite the split of train & validation sets 

In [1]:

import tensorflow as tf

import imp
imp.find_module('object_detection')
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util
import xml.etree.ElementTree as ET

from os import listdir
from os.path import isfile, join
import random

DATA_DIR = "E:\\UAV_drone\\data\\object_detection\\training"
train_record_path = join(DATA_DIR,"train.record")
val_record_path = join(DATA_DIR,"val.record")
annotation_set_dir = join(DATA_DIR, "annotations")
image_set_dir = join(DATA_DIR, "images")
label_map_path = join(DATA_DIR, "label_map.pbtxt")
train_file = join(DATA_DIR, "train_set.txt")
val_file = join(DATA_DIR, "val_set.txt")

In [2]:
# print(OUTPUT_PATH)

In [3]:
# TODO(user): Write code to read in your dataset to examples variable

samples = [f for f in listdir(image_set_dir) if isfile(join(image_set_dir, f))]
random.shuffle (samples)
split_ind = int(0.75*len(samples))
train_set = samples[:split_ind]
print (len(train_set))
val_set = samples[split_ind:]
print (len(val_set))

with open(train_file, 'w') as trf:
    trf.write("\n".join(train_set))
with open(val_file, 'w') as vaf:
    vaf.write("\n".join(val_set))

750
251


In [4]:
# flags = tf.app.flags
# flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
# FLAGS = flags.FLAGS
label_map_dict = label_map_util.get_label_map_dict(label_map_path)

def create_tf_example(example):
    # TODO(user): Populate the following variables from your example.
    root = ET.parse(join(annotation_set_dir, example+".map")).getroot()
    
    height = int(root.find('size').get('height')) # Image height
    width = int(root.find('size').get('width')) # Image width
    
    filename = example.encode('utf8') # Filename of the image. Empty if image is not from file
    with tf.gfile.GFile(join(image_set_dir, example), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_image_data = encoded_jpg # Encoded image bytes
    image_format = b'jpg' # b'jpeg' or b'png'

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
             # (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)
    
    for element in root.findall('area'):
        coords = list(map(int, element.get('coords').split(",")))
        xmins.append(float(coords[0])/width)
        xmaxs.append(float(coords[2])/width)
        ymins.append(float(coords[1])/height)
        ymaxs.append(float(coords[3])/height)
        class_name = element.get('title')
        classes_text.append(class_name.encode('utf8'))
        classes.append(label_map_dict[class_name])


    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


In [5]:
def write_tfrecord(set_files, path):
    writer = tf.python_io.TFRecordWriter(path)
    for sample in set_files:
        tf_example = create_tf_example(sample)
        writer.write(tf_example.SerializeToString())
    writer.close()
    
write_tfrecord(train_set, train_record_path)
write_tfrecord(val_set, val_record_path)

In [8]:
# verify
for example in tf.python_io.tf_record_iterator(OUTPUT_PATH):
    result = tf.train.Example.FromString(example)
    print (result)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

